In [1]:
import os
import sys

sys.path.insert(0, "../../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import ARDL, VAR
from statsmodels.tsa.ardl import ardl_select_order
from sklearn.decomposition import PCA
from src.text.epu import EPU
from src.text.gui import DrawTopics
from src.tourism.ts_utils import get_adf_df
from src.google_trends import GT
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
output_folder = sys.path[0] + "outputs/text/"

country_name = [
    "samoa", "papua_new_guinea", "fiji", "vanuatu", "solomon_islands"
]

country_iso = [
    "WS", "PG", "FJ", "VU", "SB"
]

inflation_prompts = [
    "inflation", "CPI", "food price", "purchasing power", "price"
]
GoogleAPIkey = os.getenv("GoogleAPIkey")
gt = GT(GoogleAPIkey)


In [3]:
for name, iso in zip(country_name, country_iso):

    dnx = DrawTopics(inflation_prompts, iso, gt)
    dnx.trends_to_nx_object()
    words_df = dnx.result
    inflation_words = words_df[words_df.volume >= 5]["target"].unique().tolist()


    local_trends = pd.DataFrame()
    for word in inflation_words:
        r = gt.get_graph(word, iso, "2015-01")
        r_df = gt.to_df(r)
        r_df = (r_df.drop("term", axis=1).rename({"value": word.replace(" ", "_")}, axis=1))
        if local_trends.empty:
            local_trends = r_df
        else:
            local_trends = local_trends.merge(r_df, how="left", on="date")
            
    cols = local_trends.columns[~local_trends.columns.isin(["date"])]
    local_trends["trends"] = local_trends[cols].sum(axis=1)
    local_trends.to_csv(f"{output_folder}{name}/{name}_google_volume.csv", encoding="utf-8")